# Calculate APN function

### Imports

In [ ]:
import os
import numpy as np
from datetime import datetime as dt
import pandas as pd
import json 
import random

### Calculate the APN functions

Let us consider the $f: \mathbb{F}_2^n \times \mathbb{F}_2^n \rightarrow \mathbb{F}_2^n \times \mathbb{F}_2^n$ quadratic APN function, in the form $$f(x,y) = (xy, G(x,y)).$$

First, we need to get the possible parameters, to qualify our functions to be APNs. This is based on the theorems of Carlet, Zou-Pott and Taniguchi. All can be found in "On some quadratic APN functions" by Taniguchi.

### Used Theorems:

#### Carlet
Parameters: $n,i,j,s,t,u,v$ (or $m=i-j$)

Let $n$ be an integer with $n\geq 2$ and $i,j$ integers such that $GCD(n, i-j)=1$. Let $s, t, u, v\in \mathbb{F}_{2^n}$ with the condition that $s,t\neq 0$. Set 
    $$G(x, y) = sx^{2^i+2^j}+ux^{2^i}y^{2^j} + vx^{2^j}y^{2^i} + ty^{2^i+2^j}$$
    Then $F(x,y):=(xy, G(x,y))$ is an APN function if and only if the polynomial $$G(x, 1) = sx^{2^i+2^j} + ux^{2^i}+vx^{2^j}+t$$ has no root in $\mathbb{F}_{2^n}$

#### Reduced Carlet

The function $G(x, 1) = sx^{2^i+2^j} + ux^{2^i}+vx^{2^j}+t$ has no root over $\mathbb{F}_{2^n}$ if and only if the degree of $GCD(x^{2^i+2^j} + x^{2^j} + t, x ^ {(2 ^ n)} - x)$ is zero.

#### Zou-Pott (Theorem 2)
Parameters: $n,m,\alpha,\sigma=2^i$

Let $m,n$ be positive integers with $n\geq 2 $ even and $GCD(m,n)=1$. 
    Let $\alpha \in \mathbb{F}_{2^n}^{\times}$ and set $\sigma \in Gal(\mathbb{F}_{2^n} / \mathbb{F}_{2})$, where $Gal(\mathbb{F}_{2^n} / \mathbb{F}_{2})$ is the Galois group of $\mathbb{F}_{2^n}$ over $\mathbb{F}_{2}$. Let $$G(x,y)=x^{2^m+1}+\alpha y^{(2^m+1)\sigma}.$$
    Then $F(x,y):=(xy, G(x,y))$ is an APN function if and only if 
    $$\alpha \notin \{a^{2^m+1}(t^{2^m}+t)^{1-\sigma}|a,t\in \mathbb{F}_{2^n}\}.$$

#### Zou-Pott (Corollary 1)
Let $m, n$ be positive integers, $n \geq 2$ even and
    $GCD(m, n) = 1$. Let $\alpha \in \mathbb{F}_{2^n}^\times$ and $$G(x, y) = x^{2^m+1}+\alpha y^{(2^m+1)2^i}.$$ If $\alpha$ is non-cubic (
    $\alpha \neq t^3, \forall t\in \mathbb{F}_{2^n}$) and $i$ even, then $F(x, y) := (xy, G(x, y))$ is an APN function.

#### Taniguchi (Theorem 3)
Parameters: $n,m,a,b$

Let $m, n$ be positive integers with $n \leq 2$ and $GCD(m, n) = 1$. Let $a\in \mathbb{F}_{2^n}$ and
$b\in \mathbb{F}_{2^n}^x$. Let us define
$$G(x, y) = x^{2^{2^m} +2^{3^m}}+ ax^{2^{2m}} y^{2^m} + b y^{2^m +1}$$
Then $F(x, y) := (x y, G(x, y))$ is an APN function if and only if the polynomial $$P(x) :=
x^{2^m +1} + ax + b$$ has no root in $\mathbb{F}_{2^n}$

In [ ]:
# Theorem 3
def theorem_3(n: int) -> list:
    '''
    Calculates every possible parameterset (m, 1, b) based on Taniguchi's theorem for the given n.
    
    :param int n: power of F_2 field
    return list: list of possible parameter sets
    '''
    # we may assume a=1 by Proposition 2
    x = PolynomialRing(GF(2^n), 'x').gen()
    list_of_parameters = []
    mult_group = [e for e in GF(2 ^ n) if e != 0]

    for m in range(n):
        if gcd(n, m) != 1:
            continue
        for b in mult_group:
            p = x ^ (2 ^ m + 1) + 1 * x + b
            if p.roots() == []:
                parameterset = [m, 1, b]
                list_of_parameters.append(parameterset)
    return list_of_parameters

# Corollary 1 with i=2
def corollary_1(n: int) -> list:
    '''
    Calculates every possible parameterset (alpha, m) based on Zou-Pott theorem and the corollary 1 for the given n.
    
    :param int n: power of F_2 field
    return list: list of possible parametersets
    '''
    cubes = [t ^ 3 for t in GF(2 ^ n)]
    mult_group = [e for e in GF(2 ^ n) if e != 0]
    m_values = [m for m in range(1, n) if gcd(m,n) == 1]
    params = []

    P.<x,y> = PolynomialRing(GF(2 ^ n),2,order='lex')

    for alpha in mult_group:
        for m in m_values:
            i = 2
            if alpha not in cubes and i % 2 == 0:
                params.append([alpha, m])
    return params

# Carlet
def gen_carlet_params(n: int) -> list:
    '''
    Calculates every possible parameterset (m, s, u, v, t) based on Carlet's theorem for the given n.
    
    :param int n: power of F_2 field
    return list: list of possible parametersets
    '''
    x = PolynomialRing(GF(2^n), 'x').gen()
    list_of_parameters = []

    for m in range(n):
        relative_prime = gcd(n, m) == 1
        if not relative_prime:
            continue
        for t in GF(2 ^ n):
            s, u, v = 1, 0, 1
            carlet_g1 = s * x ^ (2 ^ m + 1) + u * x ^ (2 ^ m) + v * x + t
            if gcd(carlet_g1, x ^ (2 ^ n) - x).degree() == 0:
                parameterset = [m, s, u, v, t]
                list_of_parameters.append(parameterset)
    return list_of_parameters

Our quadratic APN functions look like this: $f(x, y) = (xy, G(x, y))$
After choosing the parameters, we can calculate the the image of the APN functions. First, let us calculate the $G(x,y)$ for given parameters, $x$ and $y$.

In [ ]:
def G_theorem_3(parameters: list, x, y, n: int):
    '''
    Calculates G(x,y) for given parameterset and field. 
    :param list parameterset: is a triple [m,a,b], implying the parameters in Taniguchi's theorem.
    :param x: element of GF(2^n)
    :param y: element of GF(2^n)
    :param int n: power of F_2 field
    return: an element of GF(2^n), G(x,y)
    '''
    m, a, b = parameters

    g =  x ^ (2 ^ (2 * m) + 2 ^ (3 * m)) + a * x ^ (2 ^ (2 * m)) * y ^ (2 ^ m) + b * y ^ (2 ^ m + 1)

    return g

def G_corollary_1(parameters: list, x, y, n: int):
    '''
    Calculates G(x,y) for given parameterset and field. 
    :param list parameterset: [aplha, b], implying the parameters in Zou-Pott's corollary.
    :param x: element of GF(2^n)
    :param y: element of GF(2^n)
    :param int n: power of F_2 field
    return: an element of GF(2^n), G(x,y)
    '''

    alpha = parameters[0]
    m = parameters[1]

    g =  x ^ (2 ^ m + 1) + alpha * y ^ ((2 ^ m + 1) * 2 ^ 2)

    return g

def G_carlet(parameters: list, x, y, n: int):
    '''
    Calculates G(x,y) for given parameterset and field. 
    :param list parameterset: is a triple [m, s, u, v, t], implying the parameters in Carlet's theorem.
    :param x: element of GF(2^n)
    :param y: element of GF(2^n)
    :param int n: power of F_2 field
    return: an element of GF(2^n), G(x,y)
    '''

    m = parameters[0]
    s = parameters[1]
    u = parameters[2]
    v = parameters[3]
    t = parameters[4]


    carlet_g = s * x ^ (2 ^ m + 1) + u * x ^ (2 ^ m) + v * x * y ^ (2 ^ m) + t * y ^ (2 ^ m + 1)

    return carlet_g

In [ ]:
quadratic_apn = {'carlet': [gen_carlet_params, G_carlet], 
                 'corollary 1': [corollary_1, G_corollary_1], 
                 'theorem 3': [theorem_3, G_theorem_3]}

We create a pd.DataFrame with $5$ columns: $(x,y,x\cdot y,G(x, y),irep)$ , where 'irep' is the integer representation of the graph element $$(x,y,x\cdot y,G(x,y)) \in \mathbb{F}_2^{4n}$$ of the APN function.

In [ ]:
def create_df(function: str, n: int, how: str = 'random', paramset: list = None) -> pd.DataFrame:
    '''
    Creates the dataframe containing the x, y, xy, G(x,y) values and the integer representation of the graph 
        in F_2^(4n). Also it saves the dataframe to a csv file
    
    :param str function: Name of the quadratic APN function ('carlet', 'corollary 1', 'theorem 3')
    :param int n: The power of the of the F_2 field of the x, y, xy, G(x,y) elements
    
    :param bool randomize: Defaults to True. If True, the parameterset is chosen randomly. 
        If False then the given parameter is used.
        
    :param str how: If 'random' then a random APN with given name and dimension is computed. 
        If 'first' then the first parameter is taken from the list. If 'custom' then the given parameter is used.
        
    :param list paramset: If randomize is False this should be given. It is a list of the parameters,
        must be compatible with the function.
    return pd.DataFrame: the dataframe containing the x, y, xy, G(x,y) values and the 
        integer representation of the graph in F_2^(4n)
    '''
    # Select parameter generator and G() based on the name of the APN function
    parameter_generator = quadratic_apn[function][0]
    G_function = quadratic_apn[function][1]

    # Select the parameterset from the list of parametersets        
    if how == 'random':
        paramset = random.choice(parameter_generator(n))
    elif how == 'first':
        paramset = parameter_generator(n)[0]        
    
    # Create the dataframe
    df = pd.DataFrame(columns=['x','y','xy','Gxy','irep'])
    for x in GF(2^n):
        for y in GF(2^n):
            irep = x.integer_representation()*2^(3*n)+y.integer_representation()*2^(2*n)+(x*y).integer_representation()*2^n+G_function(paramset, x, y, n).integer_representation()
            new_row = pd.Series({
                'x': x, 
                'y': y, 
                'xy': x*y, 
                'Gxy': G_function(paramset, x, y, n),
                'irep': irep
            })
            df = pd.concat([df, new_row.to_frame().T], ignore_index=True)

    # Save the dataframe in csv
    # filename = str('Gframe__function_' + str(function) + '_n=' + str(n) + '_.csv')
    # df.to_csv(filename)
    
    return df

In [ ]:
create_df('carlet',3)

# Calculate the $\Gamma$- ranks

In [ ]:
# The elements of s_set are n-bit integers in [0..2^n-1]
def gammarank(s_set, n):
    mtx = matrix(GF(2),2^n,2^n,0)
    
    # Update the matrix
    for i in range(2^n):
        for s in s_set:
            mtx[i,i^^s] = 1

    return mtx.rank()

In [ ]:
n=4
start = dt.now()

df = create_df('carlet',n)
s_set = list(df['irep'])
rk = gammarank(s_set,4*n)

end = dt.now()

print(end-start)
print(rk)

In [ ]:
def apn_gammaranks(function, n, how = "random", parameterset: list = None, newfile: bool = False) -> int:
    '''
    Calculates the gamma rank of the APN function
    
    :param str function: Name of the quadratic APN function ('carlet', 'corollary 1', 'theorem 3')
    :param int n: The power of the of the F_2 field of the x, y, xy, G(x,y) elements
    :param str how: If 'random' then a random APN with given name and dimension is computed. 
        If 'first' then the first parameter is taken from the list. If 'custom', a parameterset needs to be given. 
        Defaults to 'random'.
    :param list parameterset: Used only if how == 'custom'.
    :param bool newfile: If True forces to recreate the file. It is adviced to do only if the parameterset
        is different or important, as that information is not coded in the filename. Defaults to False.
    
    return int: The gamma rank of the APN function (over F_2 field)
    '''
    
    filename = str('Gframe__function_' + str(function) + '_n=' + str(n) + '_.csv')
    if not os.path.isfile(filename) or newfile:
        create_df(function, n, how, parameterset) # takes random parameters with the given name and dimension
    
    # Get the s set (graph of the APN function)
    Gdf = pd.read_csv(filename, index_col=0)
    s_set = list(Gdf['irep'])
    
    return gammarank(s_set,4*n)

In [ ]:
apn_gammaranks('corollary 1', 2)